In [1]:
from notebook.services.config import ConfigManager
cm = ConfigManager()

#cm.update('livereveal', {'width': 1000, 'height': 600,'scroll': True,})
cm.update('livereveal', {'scroll': True,})

%load_ext autoreload
%autoreload 2

import sys
sys.path.append(r"C:\Users\rustr\workspace\teaching\ITA_PhD-course\modules\module6")
from viewer import MeshCatViewer

import webcolors
print(webcolors.html4_names_to_hex)
#webcolors.name_to_hex(name, spec='css3')

{'aqua': '#00ffff', 'black': '#000000', 'blue': '#0000ff', 'fuchsia': '#ff00ff', 'green': '#008000', 'gray': '#808080', 'lime': '#00ff00', 'maroon': '#800000', 'navy': '#000080', 'olive': '#808000', 'purple': '#800080', 'red': '#ff0000', 'silver': '#c0c0c0', 'teal': '#008080', 'white': '#ffffff', 'yellow': '#ffff00'}


In [2]:
%%html
<style>
body.rise-enabled div.code_cell{
    font-size:40%;
}
</style>

## Coordinate frames

* World coordinate frame (WCF)

* Robot coordinate frame (RCF)

* Tool0 coordinate frame (T0CF)

* Tool coordinate frame (TCF)

* Object coordinate frame (OCF)

<img src="https://gramaziokohler.github.io/compas_fab/latest/_images/02_coordinate_frames.jpg"  />

# Kinematic model

A kinematic model describes the motion of a robot in mathematical form without considering the forces that affect 
motion and concerns itself with the geometric relationship between elements.

<img src="https://gramaziokohler.github.io/compas_fab/latest/_images/01_robot_links_and_joints.jpg" />

In the kinematic model of a robot, the connection of different manipulator joints is known as link, and the 
integration of two or more links is called a joint. This kinematic model can be represented as a tree structure.
The tree describes the kinematic chain, i.e., the connection of robotic links with joints, and the
inter-dependendencies of these links.
This tree structure plus the underlying geometric information can be defined in Unified Robot Description Format (URDF),
which describes any robot. If the robot is mounted on external axes, these links and joints
can be added as well.

### Example UR5 URDF:

https://github.com/ros-industrial/universal_robot/blob/kinetic-devel/ur_description/urdf/ur5.urdf.xacro

## Links

Robot links are solid mechanical elements (geometry).

## Joints

The joints are the elements in a robot which helps the links to travel in different kind of movements.
The three major types of joints are:

* Revolute: A hinge joint that rotates along the axis and has a limited range specified by the upper and lower limits.

* Prismatic: A sliding joint that slides along the axis, and has a limited range specified by the upper and lower limits.

* Fixed: Not really a joint because it cannot move, all degrees of freedom are locked.


In [11]:
from compas.robots import Joint

print(Joint.REVOLUTE)
print(Joint.PRISMATIC)
print(Joint.FIXED)

0
2
3


In [ ]:
from math import pi
from compas_fab.robots import Configuration

values = [1.5, pi]
types = [Joint.PRISMATIC, Joint.REVOLUTE]

config = Configuration(values, types)
config = Configuration.from_revolute_values([pi/2, 0., 0., pi/2, pi, 0])
config = Configuration.from_prismatic_and_revolute_values([8.312], [pi/2, 0., 0., 0., 2*pi, 0.8])


In [12]:
from math import pi
from compas_fab.robots import Configuration
values = [1.5, pi]
types = [Joint.PRISMATIC, Joint.REVOLUTE]
config = Configuration(values, types)

In [ ]:
All rigid body movements are rotations, translations, or combinations of the two.

In [6]:
from compas.robots import RobotModel
from compas.robots import Link
from compas.robots import Joint
from compas.geometry import Frame
from compas.geometry import Transformation
from compas.robots.model.geometry import Origin
from compas.robots.model.geometry import Geometry
from compas.robots.model.link import Visual
from compas.robots.model.geometry import Sphere
from compas.robots.model.joint import Axis
import compas.geometry
from compas.datastructures import Mesh
from compas.geometry import Polyhedron

from compas_fab.artists import BaseRobotArtist

from compas_fab.robots import Configuration

class RobotArtist(BaseRobotArtist):

    def __init__(self, robot):
        super(RobotArtist, self).__init__(robot)

    def transform(self, geometry, transformation):
        geometry.transform(transformation)

    def draw_geometry(self, geometry, color=None):
        return geometry


link0 = Link("sun_link", visual=[Visual(Geometry(sphere=Sphere(2.5)))])
joint0 = Joint("sun_joint", 'continuous', "sun_link", "earth0_link", origin=Origin.worldXY(), axis=Axis('0 0 1'))

link1 = Link("earth0_link")
earth_origin = Origin((10, 0, 0), (1, 0, 0), (0, 1, 0))
joint1 = Joint("earth0_joint", 'continuous', "earth0_link", "earth_link", origin=earth_origin, axis=Axis('0 0 1'))

link2 = Link("earth_link", visual=[Visual(Geometry(sphere=Sphere(1)))])
earth_origin = Origin((5, 0, 0), (1, 0, 0), (0, 1, 0))
joint2 = Joint("earth_joint", 'continuous', "earth_link", "moon_link", origin=earth_origin, axis=Axis('0 0 1'))

link3 = Link("moon_link", visual=[Visual(Geometry(sphere=Sphere(0.5)))])

links = [link0, link1, link2, link3]
joints = [joint0, joint1, joint2]

sun_system = RobotModel("sun_system", joints=joints, links=links)
print(sun_system)

print(sun_system.root.visual)

artist = RobotArtist(sun_system)
for frame in sun_system.frames:
    print(frame)

names = ["sun_joint", "earth0_joint", "earth_joint"]
#configuration = Configuration([0.5, 0.5, 0.5], [Joint.CONTINUOUS] * 3)

#artist.update(configuration, names, visual=True, collision=False)

for geo in artist.draw_visual():
    print(geo)

viewer = MeshCatViewer()

for sphere in artist.draw_visual():
    print(viewer.draw_sphere(sphere, color=0xaa00ff))
viewer.jupyter_cell()


Robot name=sun_system, Links=4, Joints=3 (3 configurable)
Frame(Point(10.000, 0.000, 0.000), Vector(1.000, 0.000, 0.000), Vector(-0.000, 1.000, 0.000))
Frame(Point(15.000, 0.000, 0.000), Vector(1.000, 0.000, 0.000), Vector(-0.000, 1.000, 0.000))
Sphere(Point(0.000, 0.000, 0.000), 2.5)
Sphere(Point(10.000, 0.000, 0.000), 1.0)
Sphere(Point(15.000, 0.000, 0.000), 0.5)
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7011/static/
None
None
None


In [8]:
from compas.robots import *
from compas.datastructures.mesh import mesh_transform_numpy
from compas_fab.artists import BaseRobotArtist

class RobotArtist(BaseRobotArtist):

    def __init__(self, robot):
        super(RobotArtist, self).__init__(robot)

    def transform(self, geometry, transformation):
        mesh_transform_numpy(geometry, transformation)

    def draw_geometry(self, geometry, color=None):
        return geometry

r = 'ros-industrial/abb'
p = 'abb_irb6600_support'
b = 'kinetic-devel'

github = GithubPackageMeshLoader(r,p,b)
urdf = github.load_urdf('irb6640.urdf')

model = RobotModel.from_urdf_file(urdf)
model.load_geometry(github)

artist = RobotArtist(model)

viewer = MeshCatViewer()

for mesh in artist.draw_visual():
    viewer.draw_mesh(mesh, color=0xaa00ff)
viewer.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7012/static/


In [ ]:
import time
from roslibpy import Topic
from compas_fab.backends import RosClient

client = RosClient()
client.run()
topic = Topic(client, '/messages', 'std_msgs/String')
topic.advertise()

while True:
    topic.publish(dict(data='Hello world'))
    time.sleep(1)

client.terminate()

## ROS Protocol: Message types

* Dictionary-like data structure
* Defines the type of topics
* Large collection of predefined types

In [13]:
from compas_fab.backends.ros import String

str_msg = String('hello world')
str_msg.msg

{'data': 'hello world'}

In [14]:
from compas.geometry import Frame
from compas_fab.backends.ros import Pose

frame = Frame.worldXY()
pose = Pose.from_frame(frame)
pose.msg

{'position': {'x': 0.0, 'y': 0.0, 'z': 0.0},
 'orientation': {'x': 0.0, 'y': 0.0, 'z': 0.0, 'w': 1.0}}